**Learning FourSquare API with Python**

We will learn in details how to make calls to the Foursquare API for different purposes. We will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, we will learn how to use the visualization library, Folium, to visualize the results.

**Importing libraries**

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


**Define Foursquare Credentials and Version**

**Make sure that you have created a Foursquare developer account and have your credentials handy**

In [2]:
CLIENT_ID = 'OBRLEJ4AYCPBM3I35VGXMDOKI3OEGZAL0M0HIKK1FYUARJZK' # your Foursquare ID
CLIENT_SECRET = 'ZIKV32D2HQZN3IJM5STB0CX55FFZW2ZF0AUD3DDBUAQNMSDK' # your Foursquare Secret
ACCESS_TOKEN = 'H51LTDOHQ543LUSQSIAM124N0S1D10Y2DDX40KGZG1EIQ1DJ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OBRLEJ4AYCPBM3I35VGXMDOKI3OEGZAL0M0HIKK1FYUARJZK
CLIENT_SECRET:ZIKV32D2HQZN3IJM5STB0CX55FFZW2ZF0AUD3DDBUAQNMSDK


**Let's assume that we are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.**

In order to define an instance of the geocoder, we need to define a user_agent. 

In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


**Search for a specific venue category**

Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel.

In [4]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


**Define the corresponding URL**

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OBRLEJ4AYCPBM3I35VGXMDOKI3OEGZAL0M0HIKK1FYUARJZK&client_secret=ZIKV32D2HQZN3IJM5STB0CX55FFZW2ZF0AUD3DDBUAQNMSDK&ll=40.7149555,-74.0153365&oauth_token=H51LTDOHQ543LUSQSIAM124N0S1D10Y2DDX40KGZG1EIQ1DJ&v=20180604&query=Italian&radius=500&limit=30'

**Send the GET Request and examine the results**

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c9f9503cac2b236f829892'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
  

**Get relevant part of JSON and transform it into a pandas dataframe**

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-7-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1623849296,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1623849296,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


**Define information of interest and filter dataframe**

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",4f3232e219836c91c7bfde94


**Let's visualize the Italian restaurants that are nearby**

In [9]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
Name: name, dtype: object

**Visualizing the resturants**

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

**Exploring a Given Venue**

Let's explore the closest Italian restaurant -- Harry's Italian Pizza Bar

In [11]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=OBRLEJ4AYCPBM3I35VGXMDOKI3OEGZAL0M0HIKK1FYUARJZK&client_secret=ZIKV32D2HQZN3IJM5STB0CX55FFZW2ZF0AUD3DDBUAQNMSDK&oauth_token=H51LTDOHQ543LUSQSIAM124N0S1D10Y2DDX40KGZG1EIQ1DJ&v=20180604'

**Send GET request for result**

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '+12126081007', 'formattedPhone': '+1 212-608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
  

**Getting the venue's overall rating**

In [13]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.2


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [14]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, let's check the third restaurant.

In [15]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


**Getting the number of tips**

In [16]:
result['response']['venue']['tips']['count']

19

**Create URL and send GET request. Make sure to set limit to get all tips**

In [17]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c9fb0d8d670a4d581c4af3'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 18,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'El-Tawil',
      'gender': 'male',
      'address': '',
      'city': '',
      'state': '',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_64VpAzxd_1j4zzJ30WhPZsmHMDuV

**Getting tips and listing of associated features**

In [18]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

**Formatting column width and displaying all tips**

In [19]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

<ipython-input-19-252e55242536>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-19-252e55242536>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  tips_df = json_normalize(tips) # json normalize tips


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,484542633


**Searching a Foursquare User**

**Define URL, send GET request and display features associated with user**

In [20]:
idnumber = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}/?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber,CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN,VERSION) # define URL

# send GET request
results = requests.get(url).json()

user_data=results['response']['user']['photos']['items']

#results
pd.set_option('display.max_colwidth', -1)

users_df = json_normalize(user_data)
#This mainly used later to display the photo of the user
filtered_columns = ['id','prefix','suffix','width','height']
tips_filtered = users_df.loc[:, filtered_columns]
#url
tips_filtered

<ipython-input-20-95a321e4199e>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-20-95a321e4199e>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  users_df = json_normalize(user_data)


,id,prefix,suffix,width,height
0,5e41a77864fa070008131752,https://fastly.4sqi.net/img/general/,/484542633_ELnUC1di2LwJTjPi04McysQZNqJHSCCSxS3i_GKGTEY.jpg,1440,1440


In [21]:
g=tips_df.loc[tips_df['user.id'] == '484542633']
print('First Name: ' + tips_df['user.firstName'])
print('Last Name: ' + tips_df['user.lastName'])


0    First Name: Nick
Name: user.firstName, dtype: object
0    Last Name: El-Tawil
Name: user.lastName, dtype: object


**Retrieve the User's Profile Image**

In [22]:
# 1. grab prefix of photo 
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/general/540x920/484542633_ELnUC1di2LwJTjPi04McysQZNqJHSCCSxS3i_GKGTEY.jpg')

So it turns out that Nick is a very active Foursquare user, with more than 250 tips.

**Get User's tips**

In [23]:
# define tips URL
user_id='484542633'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

<ipython-input-23-42201c265818>:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-23-42201c265818>:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  tips_df = json_normalize(tips)


,text,agreeCount,disagreeCount,id
0,"High quality falafel that doesn’t come from a cart on the side of the road?! Yes, it exists, and it exists at Taim. Highly recommended - even if you’re new to falafel.",6,0,5ab2771a7dc9e17930670085
1,"If you are in the West Village and you choose to go to a nationwide chain coffee shop instead of 11th Street Cafe, then you are everything that is wrong with the world. This place is AWESOME!!!",3,0,5ab1daf0da5e5617d1da3c7d
2,A+! The New Whitney is MUCH more accessible and easier to navigate than the city’s other famous museums like the Met. I’m a big fan. If you’re on a budget: there’s free admission on Friday nights,5,0,5ab1d649464d655c24fc144f
3,"I had the wings and they blew my mind. Plus, Leah (the owner) is pretty awesome, so you should definitely check it out!",1,0,5ab1d0a5c666662673a11799
4,Everything here is fantastic! I’ve ordered half the menu and never been disappointed. My favorite is the spaghetti & meatballs (only served on Sundays). The spicy sauce takes it to the next level!,6,0,5ab1cbe467f62b57ee6beaa9
5,"THE spot for breakfast in the West Village. I recommend ordering the healthy treat. If you don’t like it, then you have no soul.",5,0,5ab1af48f79faa4bc867de78
6,Get the vegan peanut butter bar. It’s like sex in your mouth.,1,0,5ab1ad2847f8767a8ca88fc1
7,"My go-to for coffee and a breakfast sandwich in the mornings. That actually undersells this place - EVERYTHING is good. Lunch, pasta, desserts, wine. I’ve never been disappointed here.",2,0,5ab1acbba6ec980645fd7512
8,This is the real deal. Stuff like this is why I moved to NYC. I bet half of the guys who play here could have gone pro. Do NOT bring your cousin Jimmy (who never played before) unless you like losing!,1,0,5ab1a316a8eb606122a3b755
9,The High Line is pretty cool. The only thing not to like is that it’s so good that everyone comes here and it gets too crowded. Check it out on a weekday morning or at night to dodge the crowds.,10,0,5ab19f6849281477a8be2dab


**Let's get the venue for the tip with the greatest number of agree counts**

In [24]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION) # define URL


# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tips']['items'][0]['venue']['name'])
print(result['response']['tips']['items'][0]['venue']['location'])

Taïm
{'address': '222 Waverly Pl', 'crossStreet': 'btwn Perry & W 11th St', 'lat': 40.735970102491926, 'lng': -74.00194019079207, 'labeledLatLngs': [{'label': 'display', 'lat': 40.735970102491926, 'lng': -74.00194019079207}, {'label': 'entrance', 'lat': 40.735887, 'lng': -74.001873}], 'postalCode': '10014', 'cc': 'US', 'neighborhood': 'West Village', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['222 Waverly Pl (btwn Perry & W 11th St)', 'New York, NY 10014', 'United States']}


**Exploring a location**

So, we just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [25]:
#defining coordinates
latitude = 40.715337
longitude = -74.008848

**Define URL**

In [26]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=OBRLEJ4AYCPBM3I35VGXMDOKI3OEGZAL0M0HIKK1FYUARJZK&client_secret=ZIKV32D2HQZN3IJM5STB0CX55FFZW2ZF0AUD3DDBUAQNMSDK&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

**Send GET request and examine results**

In [27]:
import requests

In [28]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

**Getting relevant part of JSON**

In [29]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4af5d65ff964a52091fd21e3',
  'name': 'Korin',
  'location': {'address': '57 Warren St',
   'crossStreet': 'Church St',
   'lat': 40.71482437714839,
   'lng': -74.00940425461492,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71482437714839,
     'lng': -74.00940425461492},
    {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}],
   'distance': 73,
   'postalCode': '10007',
   'cc': 'US',
   'neighborhood': 'Tribeca',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'shortName': 'Furniture / Home',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/cate

**Process JSON and convert it to a clean dataframe**

In [30]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

<ipython-input-30-b7321e367fed>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(items) # flatten JSON


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Korin,Furniture / Home Store,57 Warren St,Church St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,Tribeca,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",4af5d65ff964a52091fd21e3
1,Los Tacos No. 1,Taco Place,136 Church St,NaN,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,NaN,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",5d5f24ec09484500079aee00
2,Lekka Burger,Burger Joint,81 Warren St,btw Greenwich & West Broadway,40.715246,-74.010559,"[{'label': 'display', 'lat': 40.715246, 'lng': -74.010559}]",144,10007,US,NaN,New York,NY,United States,"[81 Warren St (btw Greenwich & West Broadway), New York, NY 10007, United States]",5dc6f6a5ea8dfb00080f6faa
3,Heyday,Spa,92 Reade St,NaN,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}, {'label': 'entrance', 'lat': 40.715654, 'lng': -74.00782}]",100,10013,US,NaN,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",57ad129c498e05b086594d72
4,Chambers Street Wines,Wine Shop,148 Chambers St,btwn West Broadway & Hudson St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}, {'label': 'entrance', 'lat': 40.715696, 'lng': -74.00988}]",88,10007,US,NaN,New York,NY,United States,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]",4adcf23cf964a520cc6221e3
5,Takahachi Bakery,Bakery,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,NaN,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",4c154c9a77cea593c401d260
6,Philip Williams Posters,Antique Shop,122 Chambers St,NaN,40.715284,-74.008781,"[{'label': 'display', 'lat': 40.71528423132827, 'lng': -74.00878093952018}, {'label': 'entrance', 'lat': 40.715188, 'lng': -74.008747}]",8,10007,US,NaN,New York,NY,United States,"[122 Chambers St, New York, NY 10007, United States]",4b747291f964a52042dd2de3
7,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,btwn Greenwich St & W Broadway,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,NaN,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",54148bc6498ea7bb8c05b70a
8,Babesta,Baby Store,56 Warren Street,btw West Broadway & Church,40.714760,-74.009280,"[{'label': 'display', 'lat': 40.71476, 'lng': -74.00928}]",73,10007,US,NaN,New York,NY,United States,"[56 Warren Street (btw West Broadway & Church), New York, NY 10007, United States]",4cc368704fcfbfb70a0cbe24
9,Takahachi,Sushi Restaurant,145 Duane St,btwn W Broadway & Church St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': 'entrance', 'lat': 40.716508, 'lng': -74.007989}]",146,10013,US,NaN,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",4a8f2f39f964a520471420e3


**Let's visualize these items on the map around our location**

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

**Exploring Trending Venues**

Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.

In [32]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c9fdbcde66f55037a409a3'},
 'response': {'venues': []}}

**Checking if any venues are trending at this time**

In [33]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [34]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now depending on when we run the above code, we might get different venues since the venues with the highest foot traffic are fetched live.

**Visualize trending venues**

In [35]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [36]:
venues_map

'Cannot generate visual as no trending venues are available at the moment!'